# Hardware
As we manipulate rather big dataframes we need >80GB of free system memory.

# Install
Need:
* LaTeX `apt-get install cm-super texlive texlive-latex-extra texlive-fonts-recommended dvipng texlive-science pandoc texlive-xetex`
* Libeigen: `apt-get install libeigen3-dev`
* Python pip: `apt-get install python3-pip` and update `pip install -U pip`
* Local stuff: `pip install -r requirements.txt`
* Misc pypi: `pip install pandas matplotlib seaborn scienceplots numpy tqdm`
* Base opt itself: `pip install ..`

In [ ]:
!apt-get update
!apt-get install -y cm-super texlive texlive-latex-extra texlive-fonts-recommended dvipng texlive-science pandoc texlive-xetex libeigen3-dev python3-pip
!pip install -U pip
!pip install -r requirements.txt
!pip install pandas matplotlib seaborn scienceplots numpy tqdm
!pip install -e .

Inside docker container run `jupyter lab --allow-root --ip=0.0.0.0` to start the notebook server

In [ ]:
import re
from zipfile import ZipFile

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scienceplots  # noqa
from tqdm import tqdm

from base_opt.utilities import eval_utils
from base_opt.utilities.file_locations import ROOT

# Set up LaTeX rendering
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r"""
    \usepackage{siunitx}
""")

# Some constants
timeout = 1200

In [ ]:
from base_opt.base_opt.BaseOptimizer import BOOptimizer

n_initial_points = BOOptimizer.best_hps['n_initial_points']

In [ ]:
import uuid

# csv dtype definition
csv_dtype = {
    'Step': int,
    # 'Action': 'numpy.ndarray',
    'Reward': float,
    'Solution': 'string',
    'Valid Solution': bool,
    'Run Time': float,
    'Fail Reason': 'string',
    'Reward Fail': float,
    'Optimizer Runtime': float,
    'Task ID': 'category',
    'Algorithm': 'category',
    'Optimizer Spec': 'string',
    'Seed': int,
    'Success': bool,
}

In [ ]:
# Alternative file locator - load mixed GNU parallels output with Set/, Space/, Alg/, Seed/ directories
files = {}
zip_file = ZipFile(ROOT.joinpath('data', 'outdir_test_1200.zip'))
for csv_file in tqdm(zip_file.namelist()):
    if not csv_file.endswith('_raw.csv'):
        print(f"Skipping {csv_file}")
        continue
    # Find Task Set, Action Space, Algorithm, Seed
    if re.search(r'(?<=Set/)(.*?)/', str(csv_file)) is None:
        if 'outdir_simple' in str(csv_file):
            task_set = 'test_simple'
        elif 'outdir_edge' in str(csv_file):
            task_set = 'test_edge'
    else:
        task_set = re.search(r'(?<=Set/)(.*?)/', str(csv_file)).group(0)[:-1]
    action = re.search(r'(?<=Space/)(.*?)/', str(csv_file)).group(0)[:-1]
    algorithm = re.search(r'(?<=Alg/)(.*?)/', str(csv_file)).group(0)[:-1]
    seed = re.search(r'(?<=Seed/)(\d)+_raw.csv', str(csv_file)).group(0)[:-8]  # Remove _raw.csv
    files[csv_file] = (task_set, action, algorithm, seed)

result_df = []
for file, (task_set, action, algorithm, seed) in (pbar := tqdm(files.items())):
    pbar.set_description(f"Processing {file}")
    df = pd.read_csv(zip_file.open(file), dtype=csv_dtype)
    df['Task Set'] = task_set
    df['Action Space'] = action
    assert df['Algorithm'].nunique() == 1
    assert df['Seed'].nunique() == 1
    assert df['Algorithm'].unique()[0] == algorithm
    assert df['Seed'].unique()[0] == int(seed)
    result_df.append(df)
    
result_df = pd.concat(result_df, ignore_index=True, axis=0)

In [ ]:
result_df.columns

In [ ]:
# Split edge case task set
result_df.loc[result_df['Task Set'] == 'test_edge', 'Task Set'] += "_" + result_df.loc[result_df['Task Set'] == 'test_edge', 'Task ID'].str.extract(r"(?<=base_opt\/edge_case\/).*((?:medium)|(?:hard))")[0]

In [ ]:
# Sanity checks
print(f"{result_df['Task Set'].unique() = }")
print(f"{result_df['Action Space'].unique() = }")
print(f"{result_df['Algorithm'].unique() = }")
print(f"{result_df['Seed'].unique() = }")

In [ ]:
result_df_clean = eval_utils.cleanup_preprocess_results(
    result_df,
    group_by=['Task Set', 'Action Space', 'Algorithm', 'Task ID', 'Seed'],)

In [ ]:
result_df_clean

In [ ]:
# Find best solution ID per task id, algorithm and action space
import pickle
best_solutions = result_df_clean[result_df_clean['Valid Solution']].sort_values('Reward', ascending=False).drop_duplicates(['Task ID', 'Algorithm', 'Action Space'])
uuid_list = {}
for alg, action in best_solutions[['Algorithm', 'Action Space']].drop_duplicates().itertuples(index=False):
    item = best_solutions[(best_solutions['Algorithm'] == alg) & (best_solutions['Action Space'] == action)]
    for taskID, reward, sol_uuid in zip(item['Task ID'].tolist(), item['Reward'].tolist(), item['Solution'].tolist()):
        uuid_list[alg, action, taskID, reward] = sol_uuid
with open(ROOT.joinpath('evaluation', 'best_solutions.pkl'), 'wb') as f:
    pickle.dump(uuid_list, f)

In [ ]:
result_df_norm_time = eval_utils.normalize_time(result_df_clean, group_by=['Task Set', 'Action Space', 'Algorithm', 'Task ID', 'Seed'], sampling_time='30s')

In [ ]:
print(f"{result_df_norm_time['Task Set'].unique() = }")
print(f"{result_df_norm_time['Algorithm'].unique() = }")

In [ ]:
# Save normalized time df
result_df_norm_time.to_csv(ROOT.joinpath('data', 'normalized_time.csv'), index=False)

In [ ]:
!zip ../data/normalized_time.zip ../data/normalized_time.csv

# Basic statistics

In [ ]:
eval_utils.print_step_count(result_df_clean)

# Convergence per task set
Can be restarted from here based on normalized_time.csv with less memory usage (few GB).

In [ ]:
# Load normalized time df
result_df_norm_time = pd.read_csv(ROOT.joinpath('data', 'normalized_time.csv'))

In [ ]:
result_df_norm_time

In [ ]:
result_df_norm_time['Algorithm'].unique()

In [ ]:
result_df_norm_time['Task Set'].unique()

In [ ]:
len(result_df_norm_time['Run Time'].unique())

In [ ]:
result_df_norm_time['Seed'].unique()

## Split Success Rate and Mean Cost

In [ ]:
from scipy.stats import bootstrap
from tqdm.notebook import tqdm

res = {}
n_resamples = 9999  #  1000
rng = np.random.default_rng(42)

for row_name, row_group in tqdm(result_df_norm_time.groupby('Action Space'), desc='Action Space'):
    for col_name, col_group in tqdm(row_group.groupby('Task Set'), desc='Task Set'):
        for hue_name, hue_group in tqdm(col_group.groupby('Algorithm'), desc='Algorithm'):
            line_prefix = f"{row_name}.{col_name}.{hue_name}."
            # Success rate
            success_df = hue_group.pivot_table(index='Run Time', columns=['Seed', 'Task ID'], values='Success Till Step')
            success_values = success_df.to_numpy(dtype=np.float64)
            success_values += 1e-9 * rng.random(success_values.shape)
            bs_success = bootstrap(success_values[np.newaxis, :], np.nanmean, n_resamples=n_resamples, axis=-1, random_state=rng)
            # Mean cost of successful steps
            cost_df = hue_group.pivot_table(index='Run Time', columns=['Seed', 'Task ID'], values='Maximum Reward')
            cost_df[cost_df == -50.] = np.nan  # Filter non-solved
            cost_values = cost_df.to_numpy(dtype=np.float64)
            cost_values += 1e-9 * rng.random(cost_values.shape)
            bs_cost = bootstrap(cost_values[np.newaxis, :], np.nanmean, n_resamples=n_resamples, axis=-1, random_state=rng)
            res[line_prefix + 'mean_cost'] = cost_df.mean(axis='columns')
            res[line_prefix + 'low_cost'] = bs_cost.confidence_interval.low
            res[line_prefix + 'high_cost'] = bs_cost.confidence_interval.high
            res[line_prefix + 'mean_success_rate'] = success_df.mean(axis='columns')
            res[line_prefix + 'low_success_rate'] = bs_success.confidence_interval.low
            res[line_prefix + 'high_success_rate'] = bs_success.confidence_interval.high
    #         break  # Stop earlier for debug
    #     break  # Stop earlier for debug
    # break  # Stop earlier for debug

In [ ]:
res

In [ ]:
idx = pd.to_datetime(pd.DataFrame(res).index).astype(int) / 1e9 / 60  # Unix time to min
df = pd.DataFrame(res)
df = abs(df)
df.index = idx
df.to_csv(ROOT.joinpath('data', 'cost_success.csv'))

In [ ]:
# Last row of df is table II -> success rate and cost at end of optimization
df_tab_t_cpu = df.iloc[-1]
# Split index - scope.set.alg.measurement
df_tab_t_cpu.index = pd.MultiIndex.from_tuples(df_tab_t_cpu.index.str.split('.').tolist(), names=['Action Space', 'Task Set', 'Algorithm', 'Measurement'])
df_tab_t_cpu

In [ ]:
import itertools
list(itertools.product(['Random', 'GA', 'BO', 'SGD'], ['mean_cost', 'high_cost', 'low_cost']))

# Create Success Rate Table

In [ ]:
df_cost_t_cpu = df_tab_t_cpu.unstack(level=['Algorithm', 'Measurement']).filter(regex='.*cost') * (-1)  # Switch high/low to low/high

df_cost_t_cpu = df_cost_t_cpu[
    itertools.product(['Random', 'GA', 'BO', 'SGD'], ['mean_cost', 'high_cost', 'low_cost'])
].reindex(
    ['test_simple', 'test_hard', 'test_realworld', 'test_edge_hard'],
level=1)
table_string = df_cost_t_cpu.to_latex(float_format='%.2f')
table_string = re.sub(r'([+-]?[0-9]*[.][0-9]+)', r'\\qty{\g<1>}{}', table_string)
table_string = re.sub(r'test_simple', 'Simple', table_string)
table_string = re.sub(r'test_hard', 'Hard', table_string)
table_string = re.sub(r'test_realworld', 'Real', table_string)
table_string = re.sub(r'test_edge_medium', 'Edge Medium', table_string)
table_string = re.sub(r'test_edge_hard', 'Edge', table_string)
table_string = re.sub(r'xyz\}', r'Position}', table_string)
table_string = re.sub(r'xyz_rotvec', r'\\parbox{1.2cm}{Position + Rotation}', table_string)
table_string = re.sub(r'\[t\]', r'', table_string)
table_string = re.sub(r'{r}', r'{c}', table_string)
table_string = re.sub(r'llrrrrrrrrrrrr', r'll' + 4 * r'C@{ [}R@{, }L@{]\\hspace{4mm}}', table_string)
table_string = re.sub(r'\\cline{1-14}', r'\\midrule', table_string)
table_string = re.sub(r' &  & low & high & low & high & low & high & low & high \\\\', '', table_string)
table_string = re.sub(r' & Algorithm', 'Action Space & Task Set', table_string)
table_string = re.sub(r'Action Space & Task Set &  &  &  &  &  &  &  &  \\\\', '', table_string)
print(table_string)
df_cost_t_cpu

In [ ]:
df_success_rate_t_cpu = df_tab_t_cpu.unstack(level=['Algorithm', 'Measurement']).filter(regex='.*success_rate') * 100
df_success_rate_t_cpu = df_success_rate_t_cpu.reindex(['test_simple', 'test_hard', 'test_realworld', 'test_edge_hard'], level=1)
df_success_rate_t_cpu = df_success_rate_t_cpu[['Random', 'GA', 'BO', 'SGD']]
table_string = df_success_rate_t_cpu.to_latex(float_format='%.2f')
table_string = re.sub(r'([+-]?[0-9]*[.][0-9]+)', r'\\qty{\g<1>}{}', table_string)
table_string = re.sub(r'test_simple', 'Simple', table_string)
table_string = re.sub(r'test_hard', 'Hard', table_string)
table_string = re.sub(r'test_realworld', 'Real', table_string)
table_string = re.sub(r'test_edge_medium', 'Edge Medium', table_string)
table_string = re.sub(r'test_edge_hard', 'Edge', table_string)
table_string = re.sub(r'xyz\}', r'Position}', table_string)
table_string = re.sub(r'xyz_rotvec', r'\\parbox{1.2cm}{Position + Rotation}', table_string)
table_string = re.sub(r'\[t\]', r'', table_string)
table_string = re.sub(r'{r}', r'{c}', table_string)
table_string = re.sub(r'llrrrrrrrrrrrr', r'll' + 4 * r'C@{ [}R@{, }L@{]\\hspace{4mm}}', table_string)
table_string = re.sub(r'\\cline{1-14}', r'\\midrule', table_string)
table_string = re.sub(r' &  & low & high & low & high & low & high & low & high \\\\', '', table_string)
table_string = re.sub(r' & Algorithm', 'Action Space & Task Set', table_string)
table_string = re.sub(r'Action Space & Task Set &  &  &  &  &  &  &  &  \\\\', '', table_string)
print(table_string)
df_success_rate_t_cpu